# 🧪 Beta-VAE: Experimentos com Disentanglement

**Tutorial de Espaço Latente - Notebook 4**

## 🎯 Objetivos
- Entender o parâmetro Beta no Beta-VAE
- Comparar diferentes valores de β
- Observar trade-off: reconstrução vs disentanglement
- Usar a classe BetaVAEComparison

## 🔬 O que é Beta-VAE?

### VAE padrão (β=1)
$$\mathcal{L} = \text{Reconstruction} + \text{KL}$$

### Beta-VAE
$$\mathcal{L} = \text{Reconstruction} + \beta \cdot \text{KL}$$

**Efeito do β:**
- **β < 1**: Melhor reconstrução, menos regularização
- **β = 1**: VAE padrão (balanceado)
- **β > 1**: Maior disentanglement, pior reconstrução

**Disentanglement** = dimensões latentes independentes e interpretáveis

In [ ]:
import torch
from src.models.vae import VAE
from src.utils.data_loader import load_mnist
from src.experiments.beta_comparison import BetaVAEComparison

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

# Carrega dados
train_loader, val_loader, test_loader = load_mnist(batch_size=128)
print(f"Data loaded: {len(train_loader.dataset)} train samples")

## 🧪 Experimento: Comparando Valores de Beta

In [ ]:
# Cria comparador
comparison = BetaVAEComparison(
    model_class=VAE,
    model_kwargs={'latent_dim': 10, 'hidden_dims': [512, 256]},
    device=DEVICE
)

print(comparison)

In [ ]:
# Testa diferentes betas
betas_to_test = [0.5, 1.0, 2.0, 4.0, 8.0]

results = comparison.compare_betas(
    betas=betas_to_test,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=20,
    learning_rate=1e-3,
    verbose=True
)

print("\n✅ Training completed for all beta values!")

## 📊 Visualização Comparativa

In [ ]:
# Plota comparação completa
comparison.plot_comparison(figsize=(18, 12))

## 🗺️ Espaços Latentes Side-by-Side

In [ ]:
# Compara espaços latentes
comparison.plot_latent_spaces(test_loader, figsize=(18, 4))

## 🖼️ Qualidade de Reconstrução

In [ ]:
# Compara reconstruções
comparison.plot_reconstructions(test_loader, n_samples=8, figsize=(15, 12))

## 📈 Tabela Resumida

In [ ]:
# Métricas em tabela
summary = comparison.get_summary_table()
print(summary.to_string(index=False))

# Exporta
comparison.export_results('beta_comparison_results.csv')
print("\n✅ Results exported to beta_comparison_results.csv")

## 🔍 Análise Detalhada: β = 4.0

In [ ]:
# Pega modelo com β=4
model_beta4 = results[4.0]['model']

from src.utils.visualization import plot_latent_grid

# Se latent_dim=2, plota grid
if model_beta4.latent_dim == 2:
    plot_latent_grid(model_beta4, n_samples=20, device=DEVICE)
else:
    print(f"Latent dim = {model_beta4.latent_dim}, grid plot only works for 2D")
    
    # Gera amostras instead
    import matplotlib.pyplot as plt
    samples = model_beta4.sample(num_samples=16, device=DEVICE)
    samples = samples.view(-1, 28, 28).cpu()
    
    fig, axes = plt.subplots(4, 4, figsize=(8, 8))
    for i, ax in enumerate(axes.flat):
        ax.imshow(samples[i], cmap='gray')
        ax.axis('off')
    plt.suptitle('Generated Samples (β=4.0)', fontweight='bold')
    plt.tight_layout()
    plt.show()

## 📊 Experimento Adicional: Annealed Beta-VAE

In [ ]:
from src.models.beta_vae import AnnealedBetaVAE
from src.utils.training import train_vae

# Cria modelo com beta annealing
annealed_vae = AnnealedBetaVAE(
    input_dim=784,
    latent_dim=10,
    hidden_dims=[512, 256],
    max_beta=4.0,
    anneal_steps=10
)

print(f"Training Annealed Beta-VAE (β: 0 → 4.0 over 10 epochs)...")

history_annealed = train_vae(
    model=annealed_vae,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=25,
    learning_rate=1e-3,
    beta=1.0,  # Will be ignored, uses internal annealing
    device=DEVICE,
    verbose=True
)

from src.utils.visualization import plot_training_history
plot_training_history(history_annealed)

## 📝 Resumo

✅ Beta controla trade-off: reconstrução vs regularização  
✅ β > 1 promove disentanglement (dimensões independentes)  
✅ β < 1 melhora qualidade de reconstrução  
✅ BetaVAEComparison facilita experimentos  
✅ Annealed Beta-VAE: aumenta β gradualmente durante treino  

**Escolha de β:**
- Geração de imagens: β = 1.0
- Disentanglement: β = 4.0 - 10.0
- Compressão: β = 0.5 - 1.0

---

## 🚀 Próximo Notebook

No **Notebook 05**, vamos explorar interativamente o espaço latente!

→→ `05_exploracao_interativa.ipynb`